In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(1, '/home/jw3514/Work/CellType_Psy/src')
from CellType_PSY import *
import scanpy as sc
HGNC, ENSID2Entrez, GeneSymbol2Entrez, Entrez2Symbol, allen_mouse_genes = LoadGeneINFO()

import requests
import SimpleITK as sitk
import pathlib

from sklearn.cluster import DBSCAN
from scipy.spatial import ConvexHull
import alphashape
from alpha_shapes.boundary import Boundary, get_boundaries

In [ ]:
def Z1Conversion(ExpMat, outname="test.z1.mat"):
    Z_mat = []
    for g, row in ExpMat.iterrows():
        tmp = ZscoreConverting(row.values)
        Z_mat.append(tmp)
    Z_mat = np.array(Z_mat)
    CT_Z1_DF = pd.DataFrame(data=Z_mat, index=ExpMat.index.values, 
                            columns=ExpMat.columns.values)
    CT_Z1_DF.to_csv(outname)
    return CT_Z1_DF

# Partition MERFISH & STR Mapping with ISH

In [ ]:
MERFISH = pd.read_csv("dat/MERFISH.cells.csv")

In [ ]:
MERFISH.shape

In [ ]:
MERFISH.columns.values

In [ ]:
set(MERFISH["donor_label"])

In [ ]:
# parcellation_organ parcellation_category parcellation_division parcellation_structure parcellation_substructure

In [ ]:
CCF_V3_ontology = pd.read_excel("/home/jw3514/Work/data/Allen_Mouse_Brain_Cell_Atlas/SuppTables/41586_2023_6812_MOESM2_ESM.xlsx",
                               sheet_name="CCFv3 ontology", skiprows=1, index_col="abbreviation")

In [ ]:
print(len(set(MERFISH["parcellation_division"].values)))
print(len(set(MERFISH["parcellation_structure"].values)))
print(len(set(MERFISH["parcellation_substructure"].values)))

In [ ]:
Merfish_STRs = list(set(MERFISH["parcellation_structure"].values))

In [ ]:
Annotated_STRs = []
Missing_STRs = []
for STR in Merfish_STRs:
    if STR in CCF_V3_ontology.index.values:
        Annotated_STRs.append(STR)
    else:
        Missing_STRs.append(STR)

In [ ]:
Missing_STRs

In [ ]:
len(Annotated_STRs)

In [ ]:
# Map to 213 STR

In [ ]:
ASD_STR_ISH_Bias = pd.read_csv("/home/jw3514/Work/ASD_Circuits/dat/Unionize_bias/Spark_Meta_EWS.Z2.bias.FDR.csv", 
                               index_col=0)
Structures_ISH = []
for STR in ASD_STR_ISH_Bias.index.values:
    New_Name = STR.replace("_", " ")
    Structures_ISH.append(New_Name)

In [ ]:
for i, row in CCF_V3_ontology.iterrows():
    structure_name = row["full structure name"]
    new_name = structure_name.replace(",", "").replace("(", "").replace(")", "").replace("-", " ")
    CCF_V3_ontology.loc[i, "CleanName"] = new_name

In [ ]:
Structures_ISH = []
for STR in ASD_STR_ISH_Bias.index.values:
    New_Name = STR.replace("_", " ")
    Structures_ISH.append(New_Name)

In [ ]:
len(Structures_ISH)

In [ ]:
CCF_V3_ontology_structure_lvl = CCF_V3_ontology.loc[Annotated_STRs, :]

In [ ]:
Structure_CCF = CCF_V3_ontology_structure_lvl["CleanName"].values

In [ ]:
set(Structure_CCF).difference(set(Structures_ISH))

In [ ]:
set(Structures_ISH).difference(set(Structure_CCF))

In [ ]:
XX = ["Anteromedial nucleus", "Inferior colliculus", "Medial geniculate complex", "Medullary reticular nucleus",
     "Subiculum"]

In [ ]:
# 'Anterior area' + 'Rostrolateral visual area' = 'Posterior parietal association areas'

In [ ]:
Subiculum

In [ ]:
Subiculum = MERFISH[MERFISH["parcellation_structure"]=="AM"]
set(Subiculum["parcellation_substructure"])

In [ ]:
# Inferior colliculus IC
Subiculum = MERFISH[MERFISH["parcellation_structure"]=="IC"]
set(Subiculum["parcellation_substructure"])

In [ ]:
# Medial geniculate complex
Subiculum = MERFISH[MERFISH["parcellation_structure"]=="MG"]
set(Subiculum["parcellation_substructure"])

In [ ]:
# Medullary reticular nucleus
Subiculum = MERFISH[MERFISH["parcellation_structure"]=="MDRN"]
set(Subiculum["parcellation_substructure"])

In [ ]:
# Posterior parietal association areas
Subiculum = MERFISH[MERFISH["parcellation_structure"]=="VISa"]
print(set(Subiculum["parcellation_substructure"]))
print(set(Subiculum["parcellation_division"]))

In [ ]:
# Posterior parietal association areas
Subiculum = MERFISH[MERFISH["parcellation_structure"]=="VISrl"]
print(set(Subiculum["parcellation_substructure"]))
print(set(Subiculum["parcellation_division"]))

In [ ]:
Subiculum = MERFISH[MERFISH["parcellation_structure"]=="SUB"]
set(Subiculum["parcellation_substructure"])

Need to go substructure level in Merfish: AM(AMd AMv) IC(ICc ICd ICe) MG(MGd MGm MGv) MDRN(MDRNd MDRnv)

Need to combine structure level in Merfish: (VISa VISrl)PTLp

Need to combine structure level in ISH: Subiculum(SUB-> SUBd SUBv)

In [ ]:
#Annotate ISH STR on MERFISH data
for i, row in MERFISH.iterrows():
    _str = row["parcellation_structure"]
    _substr = row["parcellation_substructure"]
    if _str in CCF_V3_ontology.index.values:
        name_str = CCF_V3_ontology.loc[_str, "CleanName"]
    else:
        name_str = "None"
        Unidentified.append(_str)
    if _substr in CCF_V3_ontology.index.values:
        name_substr = CCF_V3_ontology.loc[_substr, "CleanName"]
        #print(_substr, name_substr)
    else:
        #print(_substr)
        name_substr = "None"
        Unidentified.append(_substr)
        
    #print(_str, name_str, _substr, name_substr)  
    
    if name_str in Structures_ISH: # should capture most of them
        ISH_STR = name_str
    elif name_substr in Structures_ISH: # should capture AM IC MG MDRN
        ISH_STR = name_substr
    elif _str in ["VISa", "VISrl"]:
        ISH_STR = "Posterior parietal association areas"
    elif name_str == "Subiculum":
        ISH_STR = "Subiculum"
    else:
        ISH_STR = "Not in Connectome"
        
    MERFISH.loc[i, "ISH_STR"] = ISH_STR

In [ ]:
#MERFISH.to_csv("dat/MERFISH.cells.AnnoISH_STR.csv", index=False)

In [ ]:
len(list(set(MERFISH["ISH_STR"])))

In [ ]:
set(Structures_ISH).difference(set(MERFISH["ISH_STR"]))

In [ ]:
set(set(MERFISH["ISH_STR"])).difference(Structures_ISH)

In [ ]:
STRs = sorted(list(set(MERFISH["ISH_STR"])))

In [ ]:
#for STR in STRs:
#    if STR != "Not"
#    print(STR)

In [ ]:
MERFISH.shape

In [ ]:
MERFISH.to_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/MERFISH/MERFISH.ISH_Annot.csv", index=False)

In [ ]:
MERFISH_subsample = MERFISH.loc[np.random.choice(MERFISH.index, 30000, replace=False)]
MERFISH_subsample.to_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/MERFISH/Sample.csv", index=False)

In [ ]:
MERFISH_subsample.shape

In [ ]:
CCF_V3_ontology_selected = CCF_V3_ontology[CCF_V3_ontology["CleanName"].isin(set(MERFISH["ISH_STR"]))]

In [ ]:
CCF_V3_ontology_selected.to_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/MERFISH/CCF_V3_ISH_MERFISH.csv")

In [ ]:
### volumne vs No.cells

# Compare Exp Level of MERFISH & ISH

In [ ]:
MERFISH_DIR = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/MERFISH"
MERFISH_CellMean_Exp = pd.read_csv("{}/STR_Cell_Mean_DF.csv".format(MERFISH_DIR), index_col=0)
MERFISH_VolMean_Exp = pd.read_csv("{}/STR_Vol_Mean_DF.csv".format(MERFISH_DIR), index_col=0)

In [ ]:
def Z1Conversion(ExpMat, outname="test.z1.mat"):
    Z_mat = []
    for g, row in ExpMat.iterrows():
        tmp = ZscoreConverting(row.values)
        Z_mat.append(tmp)
    Z_mat = np.array(Z_mat)
    CT_Z1_DF = pd.DataFrame(data=Z_mat, index=ExpMat.index.values, 
                            columns=ExpMat.columns.values)
    CT_Z1_DF.to_csv(outname)
    return CT_Z1_DF
MERFISH_CellMean_Z1 = Z1Conversion(MERFISH_CellMean_Exp, "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/MERFISH/STR_Cell_Mean_DF.Z1.csv")
MERFISH_VolMean_Z1 = Z1Conversion(MERFISH_VolMean_Exp, "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/MERFISH/STR_Vol_Mean_DF.Z1.csv")

In [ ]:
MERFISH_Median_DIR = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/MERFISH_Median/"
MERFISH_Median_CellMean_Exp = pd.read_csv("{}/STR_Cell_Mean_DF.csv".format(MERFISH_Median_DIR), index_col=0)
MERFISH_Median_VolMean_Exp = pd.read_csv("{}/STR_Vol_Mean_DF.csv".format(MERFISH_Median_DIR), index_col=0)

In [ ]:
## Z2_conversion
Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/SC_STR_CellMean/"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/MERFISH/SC_STR_CellMean.Z2.csv")

In [ ]:
Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/SC_STR_VolMean/"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/MERFISH/SC_STR_VolMean.Z2.csv")

In [ ]:
## Z2_conversion
Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/SC_STR_CellMean_AllenRegionMatch/"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/MERFISH/SC_STR_CellMean.ISH.Match.Z2.csv")

In [ ]:
MERFISH_Median_CellMean_Z1 = Z1Conversion(MERFISH_Median_CellMean_Exp, "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/MERFISH_Median/STR_Cell_Mean_DF.Z1.csv")
MERFISH_Median_VolMean_Z1 = Z1Conversion(MERFISH_Median_VolMean_Exp, "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/MERFISH_Median/STR_Vol_Mean_DF.Z1.csv")

In [ ]:
## Z2_conversion
Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/SC_STR_median_CellMean_V3_ISHMatch//"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/MERFISH_Median/SC_STR_CellMean.Z2.csv")

In [ ]:
## Z2_conversion
Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/SC_STR_median_VolMean_V3_ISHMatch/"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/MERFISH_Median/SC_STR_VolMean.Z2.csv")

In [ ]:
## Z2_conversion
Z2_split_dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/Z2.Split/SC_STR_median_CellMean_V3/"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/MERFISH_Median/SC_STR_CellMean.Z2.ABCMatch.csv")

In [ ]:
gene=10434
dat1 = MERFISH_CellMean_Exp.loc[gene,:]
dat2 = MERFISH_VolMean_Exp.loc[gene,:]
plt.scatter(dat1, dat2)

In [ ]:
ISH_Exp = pd.read_csv("/home/jw3514/Work/ASD_Circuits/dat/allen-mouse-exp/JW_ExpMat.LogMean.0418.csv", index_col=0)

In [ ]:
columns = ISH_Exp.columns.values
columns = [x.replace("_", " ") for x in columns]
ISH_Exp.columns = columns

In [ ]:
ISH_Exp

In [ ]:
Shared_Genes = np.array(list(set(MERFISH_CellMean_Exp.index.values).intersection(set(ISH_Exp.index.values))))
Shared_STRs = np.array(list(set(MERFISH_CellMean_Exp.columns.values).intersection(set(ISH_Exp.columns.values))))

In [ ]:
STR = "Frontal pole cerebral cortex"
dat1 = MERFISH_CellMean_Exp.loc[Shared_Genes, STR]
dat2 = MERFISH_VolMean_Exp.loc[Shared_Genes, STR]
dat3 = ISH_Exp.loc[Shared_Genes, STR]

In [ ]:
print(spearmanr(dat1, dat3, nan_policy="omit"))
plt.scatter(dat1, dat3, s=0.1)

In [ ]:
print(spearmanr(dat2, dat3, nan_policy="omit"))
plt.scatter(dat2, dat3, s=0.1)

In [ ]:
for STR in sorted(Shared_STRs):
    dat1 = MERFISH_CellMean_Exp.loc[Shared_Genes, STR]
    dat2 = MERFISH_VolMean_Exp.loc[Shared_Genes, STR]
    dat3 = ISH_Exp.loc[Shared_Genes, STR]
    R1, P1 = spearmanr(dat2, dat3, nan_policy="omit")
    R2, P2 = spearmanr(dat2, dat3, nan_policy="omit")
    print(STR, R)

In [ ]:
G_corr_corss_STR_CellMean = []
G_corr_corss_STR_VolMean = []
G_corr_corss_STR_Cell_v_Vol = []
for G in Shared_Genes:
    dat1 = MERFISH_CellMean_Exp.loc[G, Shared_STRs]
    dat2 = MERFISH_VolMean_Exp.loc[G, Shared_STRs]
    dat3 = ISH_Exp.loc[G, Shared_STRs]
    R1, P1 = spearmanr(dat1, dat3, nan_policy="omit")
    R2, P2 = spearmanr(dat2, dat3, nan_policy="omit")
    R3, P3 = spearmanr(dat1, dat2, nan_policy="omit")
    G_corr_corss_STR_CellMean.append(R1)
    G_corr_corss_STR_VolMean.append(R2)
    G_corr_corss_STR_Cell_v_Vol.append(R3)
    #print(STR, R)

In [ ]:
plt.figure(dpi=120)
sns.kdeplot(G_corr_corss_STR_CellMean, label="ISH_CellMean")
sns.kdeplot(G_corr_corss_STR_VolMean, label="ISH_VolMean")
sns.kdeplot(G_corr_corss_STR_Cell_v_Vol, label="CellMean_VolMean")
plt.legend()
plt.xlabel("SpearmanR")

In [ ]:
GeneCorrDF = pd.DataFrame(data={"Gene":Shared_Genes, "ISH_CellMean":G_corr_corss_STR_CellMean,
                               "ISH_VolMean":G_corr_corss_STR_VolMean,
                               "Cellmean_VolMean":G_corr_corss_STR_Cell_v_Vol})
GeneCorrDF = GeneCorrDF.set_index("Gene")

In [ ]:
GeneCorrDF.head(5)

In [ ]:
#plt.scatter(GeneCorrDF["ISH_CellMean"].values, GeneCorrDF["ISH_VolMean"].values, s=0.1)
fig, ax = plt.subplots(figsize=(4,4), dpi=120)
ax.scatter(GeneCorrDF["ISH_CellMean"].values, GeneCorrDF["ISH_VolMean"].values, s=0.1)

lims = [
    np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
    np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
]

# now plot both limits against eachother
ax.plot(lims, lims, 'k-', alpha=0.75, zorder=0)
plt.xlabel("cell mean")
plt.ylabel("vol mean")

# ASD Z2 Bias with MERFISH

In [ ]:
MERFISH_Exp_Cell = pd.read_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/MERFISH/STR_Cell_Mean_DF.csv",index_col=0)
MERFISH_Exp_Vol = pd.read_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/MERFISH/STR_Vol_Mean_DF.csv", index_col=0)

MERFISH_Z1_Cell = pd.read_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/MERFISH/STR_Cell_Mean_DF.Z1.csv",index_col=0)
MERFISH_Z1_Vol = pd.read_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/MERFISH/STR_Vol_Mean_DF.Z1.csv", index_col=0)

MERFISH_Z2_Cell = pd.read_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/MERFISH/SC_STR_CellMean.Z2.csv",index_col=0)
MERFISH_Z2_Vol = pd.read_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/MERFISH/SC_STR_VolMean.Z2.csv", index_col=0)
MERFISH_Z2_Cell2 = pd.read_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/MERFISH/SC_STR_CellMean.ISH.Match.Z2.csv", index_col=0)

In [ ]:
### Fix header
head = MERFISH_Z2_Cell.columns.values
new_head = [X.replace(" ", "_") for X in head]
MERFISH_Exp_Cell.columns = new_head
MERFISH_Exp_Vol.columns = new_head
MERFISH_Z1_Cell.columns = new_head
MERFISH_Z1_Vol.columns = new_head
MERFISH_Z2_Cell.columns = new_head
MERFISH_Z2_Vol.columns = new_head
MERFISH_Z2_Cell2.columns = new_head

In [ ]:
MERFISH_Z2_Cell.to_csv("/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/MERFISH/SC_STR_CellMean.Z2.nh.csv")

In [ ]:
ASD_GW = Fil2Dict("../../ASD_Circuits/dat/Unionize_bias/Spark_Meta_EWS.GeneWeight.csv")
del ASD_GW[23126]

In [ ]:
# ASD Genes
ASD_Bias_MERFISH_Exp_Cell = AvgSTRZ_Weighted(MERFISH_Exp_Cell, ASD_GW)
ASD_Bias_MERFISH_Exp_Vol = AvgSTRZ_Weighted(MERFISH_Exp_Vol, ASD_GW)

ASD_Bias_MERFISH_Z1_Cell = AvgSTRZ_Weighted(MERFISH_Z1_Cell, ASD_GW)
ASD_Bias_MERFISH_Z1_Vol = AvgSTRZ_Weighted(MERFISH_Z1_Vol, ASD_GW)

ASD_Bias_MERFISH_Z2_Cell = AvgSTRZ_Weighted(MERFISH_Z2_Cell, ASD_GW)
ASD_Bias_MERFISH_Z2_Vol = AvgSTRZ_Weighted(MERFISH_Z2_Vol, ASD_GW)
ASD_Bias_MERFISH_Z2_Cell2 = AvgSTRZ_Weighted(MERFISH_Z2_Cell2, ASD_GW)

In [ ]:
#print(RegionDistributionsList(ASD_Bias_MERFISH_Z2_Cell.head(50).index.values))
#print(RegionDistributionsList(ASD_Bias_MERFISH_Z2_Vol.head(50).index.values))

In [ ]:
ASD_STR_ISH_Bias = pd.read_csv("/home/jw3514/Work/ASD_Circuits/dat/Unionize_bias/Spark_Meta_EWS.Z2.bias.FDR.csv", 
                               index_col=0)
#ASD_STR_ISH_Bias.loc["Subiculum_dorsal_part"]
#ASD_STR_ISH_Bias.loc["Subiculum_ventral_part"]
ASD_STR_ISH_Bias.loc["Subiculum"] = [(0.257096+0.385119)/2, "Hippocampus", 214, 1, 0, 1]
ASD_STR_ISH_Bias = ASD_STR_ISH_Bias.drop(["Subiculum_dorsal_part", "Subiculum_ventral_part"])

In [ ]:
ASD_Bias_MERFISH_Z2_Cell.head(50)

In [ ]:
ASD_Bias_MERFISH_Z2_Cell2.head(50)

In [ ]:
# ASD_STR_ISH_Bias
# ASD_Bias_MERFISH_Z2_Cell
# ASD_Bias_MERFISH_Z2_Vol

In [ ]:
BiasCorrelation(ASD_STR_ISH_Bias, ASD_Bias_MERFISH_Exp_Cell, "ISH", "MERFISH_CellMean ExpL")

In [ ]:
BiasCorrelation(ASD_STR_ISH_Bias, ASD_Bias_MERFISH_Z1_Cell, "ISH", "MERFISH_CellMean Z1")

In [ ]:
BiasCorrelation(ASD_STR_ISH_Bias, ASD_Bias_MERFISH_Z2_Cell, "ISH", "MERFISH_CellMean Z2")

In [ ]:
BiasCorrelation(ASD_STR_ISH_Bias, ASD_Bias_MERFISH_Z2_Cell2, "ISH", "MERFISH_CellMean Z2 2")

In [ ]:
BiasCorrelation(ASD_STR_ISH_Bias, ASD_Bias_MERFISH_Z2_Vol, "ISH", "MERFISH_VolMean")

In [ ]:
BiasCorrelation(ASD_Bias_MERFISH_Z2_Cell, ASD_Bias_MERFISH_Z2_Vol, "MERFISH_CellMean", "MERFISH_VolMean")

In [ ]:
BiasCorrelation(ASD_Bias_MERFISH_Z1_Cell, ASD_Bias_MERFISH_Z2_Cell, "MERFISH_CellMean Z1", "MERFISH_CellMean Z2")

In [ ]:
ASD_Bias_MERFISH_Z1_Cell.head(50)

In [ ]:
ASD_Bias_MERFISH_Exp_Cell[ASD_Bias_MERFISH_Exp_Cell["REGION"]=="Isocortex"]

In [ ]:
ASD_Bias_MERFISH_Z1_Cell[ASD_Bias_MERFISH_Z1_Cell["REGION"]=="Hippocampus"]

In [ ]:
ASD_Bias_MERFISH_Z2_Cell[ASD_Bias_MERFISH_Z2_Cell["REGION"]=="Hippocampus"]

In [ ]:
ISH_Exp = pd.read_csv("/home/jw3514/Work/ASD_Circuits/dat/allen-mouse-exp/JW_ExpMat.LogMean.0418.csv", index_col=0)

In [ ]:
ASD_ISH_ExpL = AvgSTRZ_Weighted(ISH_Exp, ASD_GW) 

In [ ]:
X = ASD_ISH_ExpL.loc["Subiculum_dorsal_part"]
Y = ASD_ISH_ExpL.loc["Subiculum_ventral_part"]
Z = [(X[0]+Y[0])/2, "Hippocampus", 214]
ASD_ISH_ExpL.loc["Subiculum"] = Z
ASD_ISH_ExpL = ASD_ISH_ExpL.drop(["Subiculum_dorsal_part", "Subiculum_ventral_part"])

In [ ]:
BiasCorrelation(ASD_ISH_ExpL, ASD_Bias_MERFISH_Exp_Cell, "ISH ExpL", "MERFISH ExpL")

In [ ]:
BiasCorrelation(ASD_STR_ISH_Bias, ASD_Bias_MERFISH_Exp_Cell, "ISH Z2", "MERFISH ExpL")

In [ ]:
23126 in ASD_GW

In [ ]:
23126 in MERFISH_Exp_Cell.index.values

In [ ]:
MERFISH_Exp_Cell.loc[23126, :]

In [ ]:
ISH_Z1 = pd.read_csv("/home/jw3514/Work/ASD_Circuits/dat/allen-mouse-exp/AllenMouse_z1_mat.0511.csv", index_col=0)
ISH_Z2 = pd.read_csv("/home/jw3514/Work/ASD_Circuits/dat/allen-mouse-exp/AllenMouseBrain_Z2bias.csv", index_col=0)

In [ ]:
ASD_ISH_Z1 = AvgSTRZ_Weighted(ISH_Z1, ASD_GW) 
X = ASD_ISH_Z1.loc["Subiculum_dorsal_part"]
Y = ASD_ISH_Z1.loc["Subiculum_ventral_part"]
Z = [(X[0]+Y[0])/2, "Hippocampus", 214]
ASD_ISH_Z1.loc["Subiculum"] = Z
ASD_ISH_Z1 = ASD_ISH_Z1.drop(["Subiculum_dorsal_part", "Subiculum_ventral_part"])

In [ ]:
ASD_ISH_Z2 = AvgSTRZ_Weighted(ISH_Z2, ASD_GW) 
X = ASD_ISH_Z2.loc["Subiculum_dorsal_part"]
Y = ASD_ISH_Z2.loc["Subiculum_ventral_part"]
Z = [(X[0]+Y[0])/2, "Hippocampus", 214]
ASD_ISH_Z2.loc["Subiculum"] = Z
ASD_ISH_Z2 = ASD_ISH_Z2.drop(["Subiculum_dorsal_part", "Subiculum_ventral_part"])

In [ ]:
BiasCorrelation(ASD_ISH_Z1, ASD_Bias_MERFISH_Z1_Cell, "ISH Z1", "MERFISH Z1")

In [ ]:
BiasCorrelation(ASD_ISH_Z1, ASD_ISH_Z2, "ISH Z1", "ISH Z2")

In [ ]:
BiasCorrelation(ASD_ISH_Z2, ASD_Bias_MERFISH_Exp_Cell, "ISH Z1", "MERFISH Z1")

In [ ]:
BiasCorrelation(ASD_ISH_Z2, ASD_Bias_MERFISH_Z1_Cell, "ISH Z1", "MERFISH Z1")

In [ ]:
BiasCorrelation(ASD_ISH_Z2, ASD_Bias_MERFISH_Z2_Cell, "ISH Z1", "MERFISH Z1")

In [ ]:
ASD_ISH_Z2[ASD_ISH_Z2["REGION"]=="Hypothalamus"]

In [ ]:
ASD_ISH_Z1[ASD_ISH_Z1["REGION"]=="Hypothalamus"]

In [ ]:
# QN Z1
QN_Z1 = quantileNormalize(MERFISH_Z1_Cell)

In [ ]:
ASD_MERFISH_Z1QN_Cell = AvgSTRZ_Weighted(QN_Z1, ASD_GW)

In [ ]:
BiasCorrelation(ASD_STR_ISH_Bias, ASD_MERFISH_Z1QN_Cell, "ISH", "MERFISH_CellMean Z1QN")

###### QN expl then Z1

In [ ]:
def Z1Conversion(ExpMat, outname="test.z1.mat"):
    Z_mat = []
    for g, row in ExpMat.iterrows():
        tmp = ZscoreConverting(row.values)
        Z_mat.append(tmp)
    Z_mat = np.array(Z_mat)
    CT_Z1_DF = pd.DataFrame(data=Z_mat, index=ExpMat.index.values, 
                            columns=ExpMat.columns.values)
    CT_Z1_DF.to_csv(outname)
    return CT_Z1_DF

In [ ]:
QN_ExpL = quantileNormalize(MERFISH_Exp_Cell)
QN_Z1 = Z1Conversion(QN_ExpL)

In [ ]:
ASD_MERFISH_Z1QN_Cell = AvgSTRZ_Weighted(QN_Z1, ASD_GW)

In [ ]:
BiasCorrelation(ASD_STR_ISH_Bias, ASD_MERFISH_Z1QN_Cell, "ISH", "MERFISH_CellMean Z1QN")

## Sibling

In [ ]:
Dir1 = "/home/jw3514/Work/ASD_Circuits/dat/CTRL/SubSampleSib_w_GeneProb/ISH_STR"
Dir2 = "/home/jw3514/Work/ASD_Circuits/dat/CTRL/SubSampleSib_w_GeneProb/MERFISH_STR"
Sib_PearsonRs = []
Sib_SpearmanRs = []
Sib_top50_bias_ISH = []
Sib_top50_bias_Merfish = []
for i in range(1000):
    Pair1 = pd.read_csv("{}/cont.bias.{}.csv".format(Dir1, i), index_col=0)
    Pair2 = pd.read_csv("{}/cont.bias.{}.csv".format(Dir2, i), index_col=0)
    Sib_top50_bias_ISH.append(Pair1.head(50)["EFFECT"].mean())
    Sib_top50_bias_Merfish.append(Pair2.head(50)["EFFECT"].mean())
    r1, r2 = GetBiasCorrelation_STR(Pair1, Pair2)
    Sib_PearsonRs.append(r1)
    Sib_SpearmanRs.append(r2)

In [ ]:
asd_r1, asd_r2 = GetBiasCorrelation_STR(ASD_ISH_Z2, ASD_Bias_MERFISH_Z2_Cell)
asd_top50 = ASD_ISH_Z2.head(50)["EFFECT"].mean()

In [ ]:
# Correlation vs Bias level
plt.figure(dpi=120, figsize=(6,4))
print(pearsonr(Sib_PearsonRs, Sib_top50_bias_ISH))
plt.scatter(Sib_PearsonRs, Sib_top50_bias_ISH, s=10, color="grey", label="siblings", alpha=0.6)
plt.scatter(asd_r1, asd_top50, color='r', marker="*", s=50, label="ASD")
plt.xlabel("STR_Bias_Corr (ISH vs MERFISH)")
plt.ylabel("Mean top50 Bias")

In [ ]:
##### Is hypothalamus over-represent in MERFISH?
Dir1 = "/home/jw3514/Work/ASD_Circuits/dat/CTRL/SubSampleSib_w_GeneProb/ISH_STR"
Dir2 = "/home/jw3514/Work/ASD_Circuits/dat/CTRL/SubSampleSib_w_GeneProb/MERFISH_STR"
Sib_Isocortex_ISH = []
Sib_Isocortex_Merfish = []
Sib_Hypothalamus_ISH = []
Sib_Hypothalamus_Merfish = []
for i in range(1000):
    Pair1 = pd.read_csv("{}/cont.bias.{}.csv".format(Dir1, i), index_col=0)
    Pair2 = pd.read_csv("{}/cont.bias.{}.csv".format(Dir2, i), index_col=0)
    Sib_Isocortex_ISH.append(Pair1[Pair1["REGION"]=="Isocortex"]["EFFECT"].mean())
    Sib_Isocortex_Merfish.append(Pair2[Pair2["REGION"]=="Isocortex"]["EFFECT"].mean())
    Sib_Hypothalamus_ISH.append(Pair1[Pair1["REGION"]=="Hypothalamus"]["EFFECT"].mean())
    Sib_Hypothalamus_Merfish.append(Pair2[Pair2["REGION"]=="Hypothalamus"]["EFFECT"].mean())

In [ ]:
plt.figure(dpi=120)
sns.kdeplot(Sib_Isocortex_ISH, label="ISH_Isocortex", color="blue")
sns.kdeplot(Sib_Isocortex_Merfish, label="MERFISH_Isocortex", color="blue", ls="--")
sns.kdeplot(Sib_Hypothalamus_ISH, label="ISH_Hypothalamus", color="red")
sns.kdeplot(Sib_Hypothalamus_Merfish, label="MERFISH_Hypothalamus", color="red", ls="--")
plt.legend()
plt.xlabel("Bias")
plt.title("SubSampledSib")

In [ ]:
Dir1 = "/home/jw3514/Work/CellType_Psy/dat/CTRL/SubSampleSib_w_GeneProb"
Sib_Isocortex = []
Sib_Hypothalamus = []
for i in range(1000):
    Pair1 = pd.read_csv("{}/cont.bias.{}.csv".format(Dir1, i), index_col=0)
    Sib_Isocortex.append(Pair1[Pair1["Supercluster"]=="Deep-layer intratelencephalic"]["EFFECT"].mean())
    Sib_Hypothalamus.append(Pair1[Pair1["Supercluster"]=="Mammillary body"]["EFFECT"].mean())

In [ ]:
ASD_CT_Bias = pd.read_csv("test.csv", index_col=0)

In [ ]:
##### Is hypothalamus CT over-represent in Siblings?
HY_GABA = ASD_CT_Bias[ASD_CT_Bias["class_id_label"]=="12 HY GABA"].index.values
IT_ET = ASD_CT_Bias[ASD_CT_Bias["class_id_label"]=="01 IT-ET Glut"].index.values
Dir = "/home/jw3514/Work/CellType_Psy/AllenBrainCellAtlas/dat/CTRL/SubSampleSib_w_GeneProb/Cluster_V3"
Sib_Isocortex2 = []
Sib_Hypothalamus2 = []
for i in range(1000):
    SibDF = pd.read_csv("{}/cont.bias.{}.csv".format(Dir, i), index_col=0)
    Sib_Isocortex2.append(SibDF.loc[IT_ET, "EFFECT"].mean())
    Sib_Hypothalamus2.append(SibDF.loc[HY_GABA, "EFFECT"].mean())

In [ ]:
plt.figure(dpi=120)
#sns.kdeplot(Sib_Isocortex_ISH, label="ISH_Isocortex", color="blue")
sns.kdeplot(Sib_Isocortex_Merfish, label="MERFISH_Isocortex", color="blue", ls="--")
#sns.kdeplot(Sib_Hypothalamus_ISH, label="ISH_Hypothalamus", color="red")
sns.kdeplot(Sib_Hypothalamus_Merfish, label="MERFISH_Hypothalamus", color="red", ls="--")

sns.kdeplot(Sib_Isocortex, label="Deep-layer intratelencephalic", color="blue")
sns.kdeplot(Sib_Hypothalamus, label="Mammillary body", color="red",)

#sns.kdeplot(Sib_Isocortex2, label="IT-ET", color="blue", ls="..")
#sns.kdeplot(Sib_Hypothalamus2, label="HY_GABA", color="red", ls='..')

plt.legend()
plt.xlabel("Bias")
plt.title("SubSampledSib")

In [ ]:
plt.figure(dpi=120)

sns.kdeplot(Sib_Isocortex, label="Deep-layer intratelencephalic", color="blue", ls="--")
sns.kdeplot(Sib_Hypothalamus, label="Mammillary body", color="red", ls="--")

sns.kdeplot(Sib_Isocortex2, label="IT-ET", color="blue", )
sns.kdeplot(Sib_Hypothalamus2, label="HY_GABA", color="red", )

plt.legend()
plt.xlabel("Bias")
plt.title("SubSampledSib")

In [ ]:
plt.figure(dpi=120)

sns.kdeplot(Sib_Isocortex_ISH, label="ISH_Isocortex", color="yellow")
sns.kdeplot(Sib_Isocortex_Merfish, label="MERFISH_Isocortex", color="yellow", ls="--")
sns.kdeplot(Sib_Hypothalamus_ISH, label="ISH_Hypothalamus", color="green", )
sns.kdeplot(Sib_Hypothalamus_Merfish, label="MERFISH_Hypothalamus", color="green", ls="--")

sns.kdeplot(Sib_Isocortex, label="Deep-layer intratelencephalic", color="blue", ls="--")
sns.kdeplot(Sib_Hypothalamus, label="Mammillary body", color="red", ls="--")

sns.kdeplot(Sib_Isocortex2, label="IT-ET", color="blue", )
sns.kdeplot(Sib_Hypothalamus2, label="HY_GABA", color="red", )

plt.legend()
plt.xlabel("Bias")
plt.title("SubSampledSib")

In [ ]:
### Check hypothalamus Z distribution vs cortex
# ISH_Z2 # MERFISH_Z2_Cell

In [ ]:
#ISH_Z2.head(2)

In [ ]:
#MERFISH_Z2_Cell.head(2)

In [ ]:
Isocortex_STRs = ASD_ISH_Z2[ASD_ISH_Z2["REGION"]=="Isocortex"].index.values
Hypothalamus_STRs = ASD_ISH_Z2[ASD_ISH_Z2["REGION"]=="Hypothalamus"].index.values

In [ ]:
ISH_IC_AllGenes = ISH_Z2.loc[:, Isocortex_STRs].mean(axis=1)
MERFISH_IC_AllGenes = MERFISH_Z2_Cell.loc[:, Isocortex_STRs].mean(axis=1)
ISH_HY_AllGenes = ISH_Z2.loc[:, Hypothalamus_STRs].mean(axis=1)
MERFISH_HY_AllGenes = MERFISH_Z2_Cell.loc[:, Hypothalamus_STRs].mean(axis=1)

In [ ]:
plt.figure(dpi=120)
sns.kdeplot(ISH_IC_AllGenes, label="ISH_Isocortex", color="blue")
sns.kdeplot(MERFISH_IC_AllGenes, label="MERFISH_Isocortex", color="blue", ls="--")
sns.kdeplot(ISH_HY_AllGenes, label="ISH_Hypothalamus", color="red")
sns.kdeplot(MERFISH_HY_AllGenes, label="MERFISH_Hypothalamus", color="red", ls="--")
plt.legend()
plt.xlabel("Z2")

In [ ]:
# Is that a sibling thing? what about randome genes? 
# Or maybe due to weight distribution?

In [ ]:
##### Is hypothalamus over-represent in MERFISH?
Dir1 = "/home/jw3514/Work/ASD_Circuits/dat/CTRL/RandomGenes/ISH_STR"
Dir2 = "/home/jw3514/Work/ASD_Circuits/dat/CTRL/RandomGenes/MERFISH_STR"
Sib_Isocortex_ISH = []
Sib_Isocortex_Merfish = []
Sib_Hypothalamus_ISH = []
Sib_Hypothalamus_Merfish = []
for i in range(1000):
    Pair1 = pd.read_csv("{}/cont.bias.{}.csv".format(Dir1, i), index_col=0)
    Pair2 = pd.read_csv("{}/cont.bias.{}.csv".format(Dir2, i), index_col=0)
    Sib_Isocortex_ISH.append(Pair1[Pair1["REGION"]=="Isocortex"]["EFFECT"].mean())
    Sib_Isocortex_Merfish.append(Pair2[Pair2["REGION"]=="Isocortex"]["EFFECT"].mean())
    Sib_Hypothalamus_ISH.append(Pair1[Pair1["REGION"]=="Hypothalamus"]["EFFECT"].mean())
    Sib_Hypothalamus_Merfish.append(Pair2[Pair2["REGION"]=="Hypothalamus"]["EFFECT"].mean())

In [ ]:
plt.figure(dpi=120)
sns.kdeplot(Sib_Isocortex_ISH, label="ISH_Isocortex", color="blue")
sns.kdeplot(Sib_Isocortex_Merfish, label="MERFISH_Isocortex", color="blue", ls="--")
sns.kdeplot(Sib_Hypothalamus_ISH, label="ISH_Hypothalamus", color="red")
sns.kdeplot(Sib_Hypothalamus_Merfish, label="MERFISH_Hypothalamus", color="red", ls="--")
plt.legend()
plt.xlabel("Bias")
plt.title("RandomGenes")

#### Bias as ASD - Sib

In [ ]:
ASD_Bias_MERFISH_Z2_Cell.head(10)

In [ ]:
def FixSubiculum(DF):
    X = DF.loc["Subiculum_dorsal_part"]
    Y = DF.loc["Subiculum_ventral_part"]
    Z = [(X[0]+Y[0])/2, "Hippocampus", 214]
    DF.loc["Subiculum"] = Z
    DF = DF.drop(["Subiculum_dorsal_part", "Subiculum_ventral_part"])
    return DF

In [ ]:
STRs = ASD_Bias_MERFISH_Z2_Cell.index.values
data = []
for i in range(1000):
    Pair1 = pd.read_csv("{}/cont.bias.{}.csv".format(Dir1, i), index_col=0)
    Pair1 = FixSubiculum(Pair1)
    Pair1 = Pair1.loc[STRs,:]
    biases = Pair1["EFFECT"].values
    data.append(biases)
data = np.array(data)
sib_bias_df_ISH = pd.DataFrame(data=data, columns=STRs)

In [ ]:
# Get Sibling MERFISH STR biases
Dir2 = "/home/jw3514/Work/ASD_Circuits/dat/CTRL/SubSampleSib_w_GeneProb/MERFISH_STR"
STRs = ASD_Bias_MERFISH_Z2_Cell.index.values
data = []
for i in range(1000):
    Pair2 = pd.read_csv("{}/cont.bias.{}.csv".format(Dir2, i), index_col=0)
    Pair2 = Pair2.loc[STRs,:]
    biases = Pair2["EFFECT"].values
    data.append(biases)
data = np.array(data)
sib_bias_df = pd.DataFrame(data=data, columns=STRs)

In [ ]:
sib_bias_df.head(2)

In [ ]:
for _str, row in ASD_Bias_MERFISH_Z2_Cell.iterrows():
    EFFECT = row["EFFECT"]
    mean_sib = sib_bias_df[_str].mean()
    Effect2 = EFFECT - mean_sib
    ASD_Bias_MERFISH_Z2_Cell.loc[_str, "EFFECT2"] = Effect2

In [ ]:
ASD_Bias_MERFISH_Z2_Cell2 = ASD_Bias_MERFISH_Z2_Cell.sort_values("EFFECT2", ascending=False)
ASD_Bias_MERFISH_Z2_Cell2.rename(columns={'EFFECT2': 'EFFECT', 'EFFECT': 'EFFECT_old'}, inplace=True)
ASD_Bias_MERFISH_Z2_Cell2["Rank"] = ASD_Bias_MERFISH_Z2_Cell2['EFFECT'].rank(ascending=False)

In [ ]:
#ASD_Bias_MERFISH_Z2_Cell2.head(50)

In [ ]:
GetBiasCorrelation_STR(ASD_STR_ISH_Bias, ASD_Bias_MERFISH_Z2_Cell2)

In [ ]:
BiasCorrelation(ASD_STR_ISH_Bias, ASD_Bias_MERFISH_Z2_Cell, "ISH", "MERFISH CellMean Sibcorrected")

In [ ]:
BiasCorrelation(ASD_STR_ISH_Bias, ASD_Bias_MERFISH_Z2_Cell2, "ISH", "MERFISH CellMean Sibcorrected")

In [ ]:
ISH_top50 = set(ASD_STR_ISH_Bias.index.values[:50])
MERFISH_top50 = set(ASD_Bias_MERFISH_Z2_Cell2.index.values[:50])
ISH_top50.intersection(MERFISH_top50)

In [ ]:
len(ISH_top50.intersection(MERFISH_top50))

In [ ]:
ASD_Bias_MERFISH_Z2_Cell.to_csv("ASD_Bias_MERFISH_Z2_Cell.csv")

In [ ]:
STR2CellNumber = pd.read_csv("STR2CellNumber.csv", index_col=0)

In [ ]:
STR2CellNumber.head(5)

In [ ]:
for i, row in STR2CellNumber.iterrows():
    mean_sib = sib_bias_df[i].mean()
    STR2CellNumber.loc[i, "MeanSibBias"] = mean_sib
    mean_sib_ISH = sib_bias_df_ISH[i].mean()
    STR2CellNumber.loc[i, "MeanSibBias_ISH"] = mean_sib_ISH

In [ ]:
HY = STR2CellNumber[STR2CellNumber["REGION"]=="Hypothalamus"]
CTX = STR2CellNumber[STR2CellNumber["REGION"]=="Isocortex"]
CB = STR2CellNumber[STR2CellNumber["REGION"]=="Cerebellum"]
MB = STR2CellNumber[STR2CellNumber["REGION"]=="Midbrain"]

In [ ]:
fig, ax = plt.subplots(1,2, dpi=120, figsize=(12,5))

X = np.log10(STR2CellNumber["STR_ncell"].values)
Y = STR2CellNumber["MeanSibBias"].values
ax[0].scatter(X,Y, color="grey")
print(spearmanr(X,Y))
m,b = np.polyfit(X, Y, 1)
print(m,b)

X1 = np.log10(HY["STR_ncell"].values)
Y1 = HY["MeanSibBias"].values
ax[0].scatter(X1,Y1, color="red", label="HY")

X2 = np.log10(CTX["STR_ncell"].values)
Y2 = CTX["MeanSibBias"].values
ax[0].scatter(X2,Y2, color="blue", label="CTX")

X2 = np.log10(CB["STR_ncell"].values)
Y2 = CB["MeanSibBias"].values
ax[0].scatter(X2,Y2, color="green", label="CB")

X2 = np.log10(MB["STR_ncell"].values)
Y2 = MB["MeanSibBias"].values
ax[0].scatter(X2,Y2, color="yellow", label="MD")

X = np.log10(STR2CellNumber["STR_ncell"].values)
Y = STR2CellNumber["MeanSibBias_ISH"].values
ax[1].scatter(X,Y, color="grey")
print(spearmanr(X,Y))
m,b = np.polyfit(X, Y, 1)
print(m,b)

X1 = np.log10(HY["STR_ncell"].values)
Y1 = HY["MeanSibBias_ISH"].values
ax[1].scatter(X1,Y1, color="red", label="HY")

X2 = np.log10(CTX["STR_ncell"].values)
Y2 = CTX["MeanSibBias_ISH"].values
ax[1].scatter(X2,Y2, color="blue", label="CTX")

X2 = np.log10(CB["STR_ncell"].values)
Y2 = CB["MeanSibBias_ISH"].values
ax[1].scatter(X2,Y2, color="green", label="CB")

X2 = np.log10(MB["STR_ncell"].values)
Y2 = MB["MeanSibBias_ISH"].values
ax[1].scatter(X2,Y2, color="yellow", label="MD")

ax[0].set_ylim(-0.3, 0.3)
ax[1].set_ylim(-0.3, 0.3)
ax[0].set_ylabel("Mean STR Bias")
ax[0].set_xlabel("Log10 Number Cells")
ax[1].set_xlabel("Log10 Number Cells")

ax[0].set_title("SC/MERFISH")
ax[1].set_title("ISH")
plt.tight_layout()
ax[0].legend()
ax[1].legend()

In [ ]:
m,b = np.polyfit(X, Y, 1)
print(m,b)

In [ ]:
spearmanr(X,Y)

In [ ]:
STR2CellNumber = STR2CellNumber.sort_values("MeanSibBias")

In [ ]:
STR2CellNumber.tail(20)

# Test Zmatch

In [ ]:
MERFISH_CellMean_Zmatch = pd.read_csv("dat/MERFISH/Z2Mat.ABCMatch.ISH_STR.CellMean.Zmatch.csv", index_col=0)
MERFISH_VolMean_Zmatch = pd.read_csv("dat/MERFISH/Z2Mat.ABCMatch.ISH_STR.VolMean.Zmatch.csv", index_col=0)

In [ ]:
MERFISH_CellMean_Zmatch.head(2)

In [ ]:
MERFISH_VolMean_Zmatch.head(2)

In [ ]:
head = MERFISH_CellMean_Zmatch.columns.values
new_head = [X.replace(" ", "_") for X in head]
MERFISH_CellMean_Zmatch.columns = new_head
MERFISH_VolMean_Zmatch.columns = new_head

In [ ]:
ASD_Bias_MERFISH_test = AvgSTRZ_Weighted(MERFISH_CellMean_Zmatch, ASD_GW)
ASD_Bias_MERFISH_test2 = AvgSTRZ_Weighted(MERFISH_VolMean_Zmatch, ASD_GW)

In [ ]:
ASD_Bias_MERFISH_test.head(10)

In [ ]:
ASD_Bias_MERFISH_test2.head(10)

In [ ]:
#ASD_Bias_MERFISH_test.head(50)

In [ ]:
BiasCorrelation(ASD_STR_ISH_Bias, ASD_Bias_MERFISH_test, "ISH", "MERFISH_CellMean ")

In [ ]:
BiasCorrelation(ASD_STR_ISH_Bias, ASD_Bias_MERFISH_test2, "ISH", "MERFISH_CellMean ")

# Test Median

In [ ]:
MERFISH_Median_CellMean_Exp = pd.read_csv("dat/MERFISH_Median/STR_Cell_Mean_DF.csv", index_col=0)
MERFISH_Median_VolMean_Exp = pd.read_csv("dat/MERFISH_Median/STR_Vol_Mean_DF.csv", index_col=0)

In [ ]:
MERFISH_Median_CellMean_Exp_qn = quantileNormalize(MERFISH_Median_CellMean_Exp)

In [ ]:
MERFISH_Median_CellMean_Exp_Z1 = Z1Conversion(MERFISH_Median_CellMean_Exp_qn)

In [ ]:
MERFISH_Median_CellMean_Exp_Z1_qn = quantileNormalize(MERFISH_Median_CellMean_Exp_Z1)

In [ ]:
head = MERFISH_Median_CellMean_Exp_Z1_qn.columns.values
new_head = [X.replace(" ", "_") for X in head]
MERFISH_Median_CellMean_Exp_Z1_qn.columns = new_head

In [ ]:
ASD_Bias_MERFISH_test = AvgSTRZ_Weighted(MERFISH_Median_CellMean_Exp_Z1_qn, ASD_GW)

In [ ]:
ASD_Bias_MERFISH_test.head(50)

In [ ]:
MERFISH_Median_CellMean_Exp_Z1 = Z1Conversion(MERFISH_Median_CellMean_Exp)
MERFISH_Median_VolMean_Exp_Z1 = Z1Conversion(MERFISH_Median_VolMean_Exp)

In [ ]:
MERFISH_Median_CellMean_Exp_Z1_v2 = pd.read_csv("dat/MERFISH_Median/STR_Cell_Mean_DF.Z1.csv", index_col=0)
MERFISH_Median_VolMean_Exp_Z1_v2 = pd.read_csv("dat/MERFISH_Median/STR_Vol_Mean_DF.Z1.csv", index_col=0)

In [ ]:
MERFISH_Median_CellMean_Exp_Z1.head(2)

In [ ]:
MERFISH_Median_CellMean_Exp_Z1_v2.head(2)

In [ ]:
#MERFISH_Median_CellMean = pd.read_csv("dat/MERFISH_Median/SC_STR_CellMean.Z2.csv", index_col=0)
#MERFISH_Median_CellMean = pd.read_csv("dat/MERFISH_Median/SC_STR_CellMean.Z2.ABCMatch.csv", index_col=0)
MERFISH_Median_CellMean = pd.read_csv("dat/MERFISH_Median/Z2Mat.ABCMatch.ISH_STR.MedianV3.CellMean.SCMatch.csv", index_col=0)
#MERFISH_Median_VolMean = pd.read_csv("dat/MERFISH_Median/SC_STR_VolMean.Z2.csv", index_col=0)
MERFISH_Median_VolMean = pd.read_csv("dat/MERFISH_Median/Z2Mat.ABCMatch.ISH_STR.MedianV3.VolMean.SCMatch.csv", index_col=0)

#MERFISH_Median_CellMean = pd.read_csv("dat/MERFISH_Median/STR_Cell_Mean_DF.Z1.csv", index_col=0)
#MERFISH_Median_VolMean = pd.read_csv("dat/MERFISH_Median/STR_Vol_Mean_DF.Z1.csv", index_col=0)

head = MERFISH_Median_CellMean.columns.values
new_head = [X.replace(" ", "_") for X in head]
MERFISH_Median_CellMean.columns = new_head
MERFISH_Median_VolMean.columns = new_head

In [ ]:
ASD_Genes = list(ASD_GW.keys())

In [ ]:
#MERFISH_Median_CellMean.loc[ASD_Genes, "Nucleus_accumbens"].hist()
#MERFISH_Median_CellMean.loc[ASD_Genes, "Field_CA1"].hist()

In [ ]:
ASD_Bias_MERFISH_Median1 = AvgSTRZ_Weighted(MERFISH_Median_CellMean, ASD_GW)
ASD_Bias_MERFISH_Median2 = AvgSTRZ_Weighted(MERFISH_Median_VolMean, ASD_GW)

In [ ]:
ASD_Bias_MERFISH_Median2.head(50)

In [ ]:
BiasCorrelation(ASD_STR_ISH_Bias, ASD_Bias_MERFISH_Median1, "ISH", "MERFISH_CellMean ExpL")

In [ ]:
BiasCorrelation(ASD_STR_ISH_Bias, ASD_Bias_MERFISH_Median2, "ISH", "MERFISH_CellMean ExpL")